# Trabalho C209 LAB L4 - 2



#### Nome: João Pedro Martins dos Santos
#### Matrícula: 309 GES

#### Vídeo Inicial = https://drive.google.com/file/d/1iWWr4oKisBgVWpGnD_ciNA_NaP0VtUWR/view?usp=drive_link
#### Vídeo Final = https://drive.google.com/file/d/1Rf34VFys1DZXDZP9k7Fz5UDzWpU2_GCS/view?usp=sharing

1 - Importar Bibliotecas

In [ ]:
import moviepy.editor as mpe
import numpy as np

2 - Carrego o vídeo e adiciono a uma variável

In [ ]:
video = mpe.VideoFileClip("inicial.mp4")

3 - Inverter a imagem, no eixo, a cada 20 segundo

In [ ]:
# função que inverte a imagem pelo numpy
def invert(array):
    return array[:, ::-1, :]

In [ ]:
# Separando em partes de 20 segundos
intervalo = 20
duracao = video.duration
num_subclips = int(duracao // intervalo)
subclips = []

In [ ]:
# Percorro os subclips preocupando em não ultrapassar 
for i in range(num_subclips):
    start = i * intervalo

    # Segurança para não ultrapassar
    end = min((i + 1) * intervalo, duracao)

    subclip = video.subclip(start, end)

    # Se o índice for ímpar inverte no eixo x
    if i % 2 == 1:
        subclip = subclip.fl_image(invert)

    subclips.append(subclip)

# Concatenação
video = mpe.concatenate_videoclips(subclips)

4 - Diminuir o som, de forma gradativa, a cada 30 segundos, de forma que os últimos 10 
segundos fique em silêncio (sem som) 

In [ ]:
# Função para diminuir volume
def fade_out(audio, t):
    frame = audio(t)

    # Número de blocos de 30 segundos -> tratando para que a divisão não arredonde para baixo
    n_blocos = int(video.duration / 30)
    if video.duration % 30 != 0:
        n_blocos += 1

    # Diminuição de volume por bloco  por exemplo se tiver 4 blocos é 25% pra cada
    volume_por_bloco = 1 / (n_blocos)

    # Em qual bloco o tempo atual está
    bloco = t / 30

    # Calcula o volume para esse bloco
    volume = 1 - (bloco * volume_por_bloco)

    frame[:, 0] *= volume
    frame[:, 1] *= volume

    return frame

# Função para mutar
def mute(audio, t):
    frame = audio(t)
    frame[:, 0] *= 0
    frame[:, 1] *= 0
    return frame

duracao = video.duration

# Separa o vídeo em duas partes
video_fade_out = video.subclip(0, duracao - 10)  # Do início até 10s antes do fim
video_mute = video.subclip(duracao - 10, duracao)  # Últimos 10 segundos

# Aplica fade out na primeira parte
audio_fade = video_fade_out.audio.fl(fade_out)
video_fade_out = video_fade_out.set_audio(audio_fade)

# Aplica mute na segunda parte
audio_mute = video_mute.audio.fl(mute)
video_mute = video_mute.set_audio(audio_mute)

# Junta as partes
video_som_modificado = mpe.concatenate_videoclips([video_fade_out, video_mute])


5 - Após o primeiro minuto, 60 segundos, fazer um corte para o segundo 80

In [ ]:
video_primeiro_minuto = video_som_modificado.subclip(0, 60)
video_cortado = video_som_modificado.subclip(60, 80)
video_segunda_parte =  video_som_modificado.subclip(80, video_som_modificado.duration)

6 - Por fim, insira os 20 segundos cortados acima no fim do video

In [ ]:
video_final = mpe.concatenate_videoclips([video_primeiro_minuto, video_segunda_parte, video_cortado])

video_final.write_videofile('resultado.mp4')